In [ ]:
import multiprocessing, cv2, os
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from libvisu import *
from ABCImaging.libimage import fetchImagesPaths
from ABCImaging.VideoManagment.videolib import generateVideoFromList
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)  # Set a wide width for display
pd.set_option('display.colheader_justify', 'center')  # Align column headers

## Configuration

In [ ]:
# Path to the folder containing the pictures
rootpath_imgs = '/Users/cyrilmonette/Library/CloudStorage/SynologyDrive-data/24.11-25.01_metabolism_OH/Images/'                         # For images
rootpath_data = 'data/2025-01-03_13-06_influxdb_data.csv'  # For CO2, htr and tmp data
hive_nb = "1" 
t_res = "1min" # Image time resolution. Change only if images are not taken every minute. Use frame_drop to keep only some frames, not this.

first_dt = "241215-120000Z"
last_dt = "241218-040000Z"
frame_drop = 10 # We keep 1 frame every frame_drop frames. Put one to keep all frames.

## Running

#### Image fetching

In [ ]:
# Get the target dt (for which we need an image, data, etc.)
datetimes = pd.date_range(start=pd.to_datetime(first_dt, format='%y%m%d-%H%M%S%z'), end=pd.to_datetime(last_dt, format='%y%m%d-%H%M%S%z'), freq=t_res)
datetimes = datetimes[::frame_drop]

imgs_paths = fetchImagesPaths(rootpath_imgs, datetimes, hive_nb)

#### tmp fetching

In [ ]:
# We fetch the thermal and other sensory data from the hive, for both RPis.
rootpath_data = os.path.abspath(rootpath_data) # Absolute path to the data
all_data = prepareData(rootpath_data)                   # All data in the csv nicely formatted
hive_data = extractData(all_data, int(hive_nb), datetimes) # Restricted to our time frame and hive
upper, lower = generateThermalDF(hive_data,datetimes)   # Only tmp data, in a format that can be used by ThermalFrame
# Seek the max and min values of the tmp data, in both upper and lower hives
max_temp = max(upper.max().max(), lower.max().max())
min_temp = min(upper.min().min(), lower.min().min())
print("Max temperature in the selected time range: ", max_temp)
print("Min temperature in the selected time range: ", min_temp)
print(upper)

#### Metabolic data fetching

In [ ]:
# Now we fetch the metabolic data
co2_data =generateMetabolicDF(hive_data)
print(co2_data)

#### htr fetching

In [ ]:
# Alternative used now:
upper_htr = hive_data[(hive_data["inhive_loc"] == "upper") & (hive_data["_measurement"] == "htr")]
lower_htr = hive_data[(hive_data["inhive_loc"] == "lower") & (hive_data["_measurement"] == "htr")]
# Drop the _measurement column
upper_htr = upper_htr.drop(columns=["_measurement"])
lower_htr = lower_htr.drop(columns=["_measurement"])
print(upper_htr)

#### Plotting

In [ ]:
vmax = max_temp
vmin = min_temp
# For a random time frame, generate the image of the hive and the thermal plot side by side with matplotlib
# This is just to check that everything is working fine
plt.figure(figsize=(18, 12))
frame = 100 # Frame within datetimes
dt = datetimes[frame]
_imgs_paths = imgs_paths.loc[dt].to_numpy()
imgs_names = [str(_imgs_paths[i]).split("/")[-1] for i in range(len(_imgs_paths))]
# Read the images using cv2
current_imgs = [cv2.imread(_imgs_paths[i], cv2.IMREAD_GRAYSCALE) if _imgs_paths[i] is not None else np.zeros((2592,4608), np.uint8) for i in range(len(_imgs_paths))]

hs_1 = Hive(current_imgs,imgs_names,ThermalFrame(upper.loc[dt].to_numpy()),ThermalFrame(lower.loc[dt].to_numpy()),co2_data.loc[dt],upper_htr.loc[dt],lower_htr.loc[dt])
contours = list(range(6, 32, 2))
snapshot = hs_1.snapshot(v_max=vmax,v_min=vmin,contours=contours,thermal_transparency=0.35)
plt.imshow(snapshot,cmap='gray')
plt.axis('off') 
plt.show()

#### Automation

In [ ]:
final_imgs = []
print("Generating frames...")
for dt in tqdm(datetimes):
    _imgs_paths = imgs_paths.loc[dt].to_numpy()
    imgs_names = [str(_imgs_paths[j]).split("/")[-1] for j in range(len(_imgs_paths))]
    # Read the images using cv2
    current_imgs = []
    for _img_path in _imgs_paths:
        if _img_path is not None:
            img = cv2.imread(_img_path, cv2.IMREAD_GRAYSCALE)
            current_imgs.append(img)
        else:
            current_imgs.append(np.zeros((2592,4608), np.uint8))
            
    hs = Hive(current_imgs, imgs_names, ThermalFrame(upper.loc[dt].to_numpy()), ThermalFrame(lower.loc[dt].to_numpy()), co2_data.loc[dt], upper_htr.loc[dt], lower_htr.loc[dt])
    snapshot = hs.snapshot(v_max=max_temp, v_min=min_temp,contours=contours,thermal_transparency=0.4)

    final_imgs.append(snapshot)

# Change frames from rgb to bgr
print("Convert frames to bgr...")
for i in tqdm(range(len(final_imgs))):
    final_imgs[i] = cv2.cvtColor(final_imgs[i], cv2.COLOR_RGB2BGR)

dest = "outputVideos/"
# Make this a global path
print("Saving video...")
generateVideoFromList(final_imgs, dest="outputVideos/", name="hive" + hive_nb + "_" + first_dt + "_" + last_dt, fps=10, grayscale=False)

# Cleanup step to release resources
multiprocessing.active_children()